- 2개의 callback

In [2]:
import dash
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go

import app_layout as al
import mylib as my

import inference as infr

import pandas as pd
import numpy as np
import sys, os
import datetime
import pickle as pkl

In [3]:
items = ['button1','time1', 'pred1', 'graph1', 'table1', 'c_time0', 'counter']
model = pkl.load(open("model.pkl","rb"))
trans = pkl.load(open("transform.pkl","rb"))

In [4]:
app = dash.Dash()
app.layout =  al.app_layout(items)

In [5]:
# 1. 자동 적재

@app.callback(
    Output('c_time0','children'),
    Input('counter', 'n_intervals'),
)
def live1(n_interval):
    df = my.db_to_df_random(db_name='steel.db', table_name='test')
    pred, x_cols = infr.inference(trans, model, df)
    pred_df = pd.DataFrame(data=pred, columns=["pred"])

    my.df_to_db(df[x_cols], "operation.db", "input_x")
    my.df_to_db(pred_df, "operation.db","pred")
    
    tt = str(datetime.datetime.now())
    return tt

In [6]:
# 클릭한 시점의 마지막 데이터를 불러오기
@app.callback(
    Output('time1','children'),
    Output('table1','data'),
    Output('pred1', 'children'),
    Output('graph1', 'figure'),
    Input('button1','n_clicks'),
    prevent_initial_call=False,
)
def fn(n_clicks): 
    
    tt = str(datetime.datetime.now())
    
    pred = my.db_last_row_to_df(db_name='operation.db', table_name='pred')['pred'].values[0]
        
    print_df = my.db_to_df("operation.db","input_x")
    pred_history = my.db_to_df("operation.db","pred")
    pred_history = pd.DataFrame(np.array([pred_history.index.tolist(),
                                          pd.to_numeric(pred_history['pred']).tolist()]).T, columns=['index','pred'])
    fig = px.line(pred_history, x='index', y='pred')    
    out=print_df.to_dict('records')
    
    return tt, out, pred, fig

In [7]:
app.run_server(host="0.0.0.0", port=9101) 

[2024-04-11 11:50:48,686] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
      